# can_intrusion_detection-Copy1


In [1]:
import warnings
warnings.filterwarnings("ignore")


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score, precision_score, recall_score, f1_score
import lightgbm as lgb
import catboost as cbt
import xgboost as xgb
import time
from river import stream
from statistics import mode


In [5]:
df1= pd.read_fwf("C:\Intel\Desktop\DoS attack datasets.txt",nrows=100)

In [6]:
df1.head(1000000)

,Timestamp:,0.000000,ID:,0000,000,DLC:,8,00,00.1,00.2,00.3,00.4,00.5,00.6,00.7
0,Timestamp:,0.000271,ID:,0080,0,DLC:,8,00,17,dc,09,16,11,16,bb
1,Timestamp:,0.000495,ID:,0000,0,DLC:,8,00,00,00,00,00,00,00,00
2,Timestamp:,0.000736,ID:,0081,0,DLC:,8,40,84,87,00,00,00,00,6b
3,Timestamp:,0.000983,ID:,0000,0,DLC:,8,00,00,00,00,00,00,00,00
4,Timestamp:,0.001239,ID:,0165,0,DLC:,8,00,08,80,00,00,00,08,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Timestamp:,0.023371,ID:,0000,0,DLC:,8,00,00,00,00,00,00,00,00
96,Timestamp:,0.023586,ID:,0329,0,DLC:,8,84,be,80,8c,11,2c,00,10
97,Timestamp:,0.023832,ID:,0000,0,DLC:,8,00,00,00,00,00,00,00,00
98,Timestamp:,0.024085,ID:,0370,0,DLC:,8,ff,20,00,80,ff,00,00,28


In [7]:
df1=df1.drop(labels=['Timestamp:','0.000000','ID:','DLC:','000','8'],axis=1)

In [8]:

dict={'0000':'CANID','00':'D0','00.1':'D1','00.2':'D2','00.3':'D3','00.4':'D4','00.5':'D5','00.6':'D6','00.7':'D7'}
df1.rename(columns=dict,inplace=True)
display(df1)

,CANID,D0,D1,D2,D3,D4,D5,D6,D7
0,0080,00,17,dc,09,16,11,16,bb
1,0000,00,00,00,00,00,00,00,00
2,0081,40,84,87,00,00,00,00,6b
3,0000,00,00,00,00,00,00,00,00
4,0165,00,08,80,00,00,00,08,80
...,...,...,...,...,...,...,...,...,...
95,0000,00,00,00,00,00,00,00,00
96,0329,84,be,80,8c,11,2c,00,10
97,0000,00,00,00,00,00,00,00,00
98,0370,ff,20,00,80,ff,00,00,28


In [9]:
df1=df1.replace(np.nan,0)

In [10]:
df1['CANID']=df1['CANID'].apply(lambda x: int(x, 16))
df1['D0']=df1['D0'].apply(lambda x: int(str(x), 16))
df1['D1']=df1['D1'].apply(lambda x: int(str(x), 16))
df1['D2']=df1['D2'].apply(lambda x: int(str(x), 16))
df1['D3']=df1['D3'].apply(lambda x: int(str(x), 16))
df1['D4']=df1['D4'].apply(lambda x: int(str(x), 16))
df1['D5']=df1['D5'].apply(lambda x: int(str(x), 16))
df1['D6']=df1['D6'].apply(lambda x: int(str(x), 16))
df1['D7']=df1['D7'].apply(lambda x: int(str(x), 16))
print(df1)

    CANID   D0   D1   D2   D3   D4  D5  D6   D7
0     128    0   23  220    9   22  17  22  187
1       0    0    0    0    0    0   0   0    0
2     129   64  132  135    0    0   0   0  107
3       0    0    0    0    0    0   0   0    0
4     357    0    8  128    0    0   0   8  128
..    ...  ...  ...  ...  ...  ...  ..  ..  ...
95      0    0    0    0    0    0   0   0    0
96    809  132  190  128  140   17  44   0   16
97      0    0    0    0    0    0   0   0    0
98    880  255   32    0  128  255   0   0   40
99      0    0    0    0    0    0   0   0    0

[100 rows x 9 columns]


In [11]:
# Z-score normalization
features = df1.dtypes[df1.dtypes != 'object'].index
df1[features] = df1[features].apply( lambda x: (x - x.mean()) / (x.std()))
print(features)
df1.head()

Index(['CANID', 'D0', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7'], dtype='object')


,CANID,D0,D1,D2,D3,D4,D5,D6,D7
0,-0.506528,-0.449318,-0.113524,2.752723,-0.301065,-0.094739,-0.002800,0.212781,3.120691
1,-0.792652,-0.449318,-0.476673,-0.509149,-0.440375,-0.422967,-0.368921,-0.351897,-0.442003
2,-0.504293,0.457250,1.607487,1.492455,-0.440375,-0.422967,-0.368921,-0.351897,1.596544
3,-0.792652,-0.449318,-0.476673,-0.509149,-0.440375,-0.422967,-0.368921,-0.351897,-0.442003
4,0.005365,-0.449318,-0.350360,1.388668,-0.440375,-0.422967,-0.368921,-0.146559,1.996632


In [12]:
df1['Label']=0

In [13]:
df1.head(100000)

,CANID,D0,D1,D2,D3,D4,D5,D6,D7,Label
0,-0.506528,-0.449318,-0.113524,2.752723,-0.301065,-0.094739,-0.002800,0.212781,3.120691,0
1,-0.792652,-0.449318,-0.476673,-0.509149,-0.440375,-0.422967,-0.368921,-0.351897,-0.442003,0
2,-0.504293,0.457250,1.607487,1.492455,-0.440375,-0.422967,-0.368921,-0.351897,1.596544,0
3,-0.792652,-0.449318,-0.476673,-0.509149,-0.440375,-0.422967,-0.368921,-0.351897,-0.442003,0
4,0.005365,-0.449318,-0.350360,1.388668,-0.440375,-0.422967,-0.368921,-0.146559,1.996632,0
...,...,...,...,...,...,...,...,...,...,...
95,-0.792652,-0.449318,-0.476673,-0.509149,-0.440375,-0.422967,-0.368921,-0.351897,-0.442003,0
96,1.015739,1.420479,2.523254,1.388668,1.726672,-0.169336,0.578687,-0.351897,-0.137173,0
97,-0.792652,-0.449318,-0.476673,-0.509149,-0.440375,-0.422967,-0.368921,-0.351897,-0.442003,0
98,1.174448,3.162790,0.028578,-0.509149,1.540925,3.381498,-0.368921,-0.351897,0.320071,0


In [ ]:
df2 = pd.read_fwf("C:/Desktop/Fuzzy attack datasets.txt",nrows=100)

In [ ]:
df2.head(1000000)

In [ ]:
df2=df2.drop(labels=['Timestamp:','0.000000','ID:','DLC:','000','8'],axis=1)

In [ ]:
dict={'0220':'CANID','29':'D0','c5':'D1','26':'D2','55':'D3','6a':'D4','67':'D5','02':'D6','5d':'D7'}
df2.rename(columns=dict,inplace=True)
display(df2)

In [ ]:
df2=df2.replace(np.nan,0)

In [ ]:
df2['CANID']=df2['CANID'].apply(lambda x: int(x, 16))
df2['D0']=df2['D0'].apply(lambda x: int(str(x), 16))
df2['D1']=df2['D1'].apply(lambda x: int(str(x), 16))
df2['D2']=df2['D2'].apply(lambda x: int(str(x), 16))
df2['D3']=df2['D3'].apply(lambda x: int(str(x), 16))
df2['D4']=df2['D4'].apply(lambda x: int(str(x), 16))
df2['D5']=df2['D5'].apply(lambda x: int(str(x), 16))
df2['D6']=df2['D6'].apply(lambda x: int(str(x), 16))
df2['D7']=df2['D7'].apply(lambda x: int(str(x), 16))
print(df2)

In [ ]:
# Z-score normalization
features = df2.dtypes[df1.dtypes != 'object'].index
df2[features] = df1[features].apply( lambda x: (x - x.mean()) / (x.std()))
print(features)
df2.head()

In [ ]:
df2['Label']=1

In [ ]:
df2.head()

In [ ]:
df3= pd.read_fwf("C:/Desktop/Impersonation attack datasets.txt",nrows=100)

In [ ]:
df3.head(1000000)

In [ ]:
df3=df3.drop(labels=['Timestamp:','1481192898.075465','ID:','DLC:','000','8'],axis=1)

In [ ]:

dict={'0587':'CANID','00':'D0','00.1':'D1','00.2':'D2','00.3':'D3','00.4':'D4','00.5':'D5','00.6':'D6','01':'D7'}
df3.rename(columns=dict,inplace=True)
display(df3)

In [ ]:
df3=df3.replace(np.nan,0)

In [ ]:
df3['CANID']=df3['CANID'].apply(lambda x: int(x, 16))
df3['D0']=df3['D0'].apply(lambda x: int(str(x), 16))
df3['D1']=df3['D1'].apply(lambda x: int(str(x), 16))
df3['D2']=df3['D2'].apply(lambda x: int(str(x), 16))
df3['D3']=df3['D3'].apply(lambda x: int(str(x), 16))
df3['D4']=df3['D4'].apply(lambda x: int(str(x), 16))
df3['D5']=df3['D5'].apply(lambda x: int(str(x), 16))
df3['D6']=df3['D6'].apply(lambda x: int(str(x), 16))
df3['D7']=df3['D7'].apply(lambda x: int(str(x), 16))
print(df3)

In [ ]:
# Z-score normalization
features = df3.dtypes[df1.dtypes != 'object'].index
df3[features] = df3[features].apply( lambda x: (x - x.mean()) / (x.std()))
print(features)
df3.head()

In [ ]:
df3['Label']=2

In [ ]:
df3.head()

In [ ]:
df4= pd.read_fwf("C:/Desktop/attack free datasets.txt",nrows=100)

In [ ]:
df4.head(10000000)

In [ ]:
df4=df4.drop(labels=['Timestamp:','0.000000','ID:','DLC:','000','8'],axis=1)

In [ ]:

dict={'0316':'CANID','05':'D0','20':'D1','ea':'D2','0a':'D3','20.1':'D4','1a':'D5','00':'D6','7f':'D7'}
df4.rename(columns=dict,inplace=True)
display(df4)

In [ ]:
df4=df4.replace(np.nan,0)

In [ ]:
df4['CANID']=df4['CANID'].apply(lambda x: int(x, 16))
df4['D0']=df4['D0'].apply(lambda x: int(str(x), 16))
df4['D1']=df4['D1'].apply(lambda x: int(str(x), 16))
df4['D2']=df4['D2'].apply(lambda x: int(str(x), 16))
df4['D3']=df4['D3'].apply(lambda x: int(str(x), 16))
df4['D4']=df4['D4'].apply(lambda x: int(str(x), 16))
df4['D5']=df4['D5'].apply(lambda x: int(str(x), 16))
df4['D6']=df4['D6'].apply(lambda x: int(str(x), 16))
df4['D7']=df4['D7'].apply(lambda x: int(str(x), 16))
print(df4)

In [ ]:
# Z-score normalization
features = df4.dtypes[df1.dtypes != 'object'].index
df4[features] = df4[features].apply( lambda x: (x - x.mean()) / (x.std()))
print(features)
df4.head()

In [ ]:
df4['Label']=3

In [ ]:
df4.head()

In [ ]:
df=pd.concat([df1,df2,df3,df4])

In [ ]:
df.head(800000)

In [ ]:
df.dtypes

In [ ]:

X = df.drop(['Label'],axis=1)
y = df['Label']
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.8, test_size = 0.2, random_state = 0)

In [ ]:
pd.Series(y_train).value_counts()

**Corresponding Attack Types:** 

0 DOS ATTACK DATASET           &emsp;                

1 FUZZY ATTACK DATASET &emsp;             

2 IMPERSONATION ATTACK DATASET &emsp;        

3 ATTACK FREE DATASET &emsp;   &emsp;      

 


In [ ]:
# from imblearn.over_sampling import SMOTE
# smote=SMOTE(n_jobs=-1,sampling_strategy={2:4000,4:4000})

In [ ]:
# X_train, y_train = smote.fit_resample(X_train, y_train)

In [ ]:
%%time
# Train the XGBoost algorithm
import xgboost as xgb
xg = xgb.XGBClassifier()

X_train_x = X_train.values
X_test_x = X_test.values

xg.fit(X_train_x, y_train)

y_pred = xg.predict(X_test_x)
print(classification_report(y_test,y_pred))
print("Accuracy of XGBoost: "+ str(accuracy_score(y_test, y_pred)))
print("Precision of XGBoost: "+ str(precision_score(y_test, y_pred, average='weighted')))
print("Recall of XGBoost: "+ str(recall_score(y_test, y_pred, average='weighted')))
print("Average F1 of XGBoost: "+ str(f1_score(y_test, y_pred, average='weighted')))
print("F1 of XGBoost for each type of attack: "+ str(f1_score(y_test, y_pred, average=None)))
xg_f1=f1_score(y_test, y_pred, average=None)

# Plot the confusion matrix
cm=confusion_matrix(y_test,y_pred)
f,ax=plt.subplots(figsize=(5,5))
sns.heatmap(cm,annot=True,linewidth=0.5,linecolor="red",fmt=".0f",ax=ax)
plt.xlabel("y_pred")
plt.ylabel("y_true")
plt.show()


In [ ]:
%%time
# Train the LightGBM algorithm
import lightgbm as lgb
lg = lgb.LGBMClassifier()
lg.fit(X_train, y_train)
y_pred = lg.predict(X_test)
print(classification_report(y_test,y_pred))
print("Accuracy of LightGBM: "+ str(accuracy_score(y_test, y_pred)))
print("Precision of LightGBM: "+ str(precision_score(y_test, y_pred, average='weighted')))
print("Recall of LightGBM: "+ str(recall_score(y_test, y_pred, average='weighted')))
print("Average F1 of LightGBM: "+ str(f1_score(y_test, y_pred, average='weighted')))
print("F1 of LightGBM for each type of attack: "+ str(f1_score(y_test, y_pred, average=None)))
lg_f1=f1_score(y_test, y_pred, average=None)


# Plot the confusion matrix
cm=confusion_matrix(y_test,y_pred)
f,ax=plt.subplots(figsize=(5,5))
sns.heatmap(cm,annot=True,linewidth=0.5,linecolor="red",fmt=".0f",ax=ax)
plt.xlabel("y_pred")
plt.ylabel("y_true")
plt.show()

In [ ]:
%%time
# Train the CatBoost algorithm
import catboost as cbt
cb = cbt.CatBoostClassifier(verbose=0,boosting_type='Plain')
#cb = cbt.CatBoostClassifier()

cb.fit(X_train, y_train)
y_pred = cb.predict(X_test)
print(classification_report(y_test,y_pred))
print("Accuracy of CatBoost: "+ str(accuracy_score(y_test, y_pred)))
print("Precision of CatBoost: "+ str(precision_score(y_test, y_pred, average='weighted')))
print("Recall of CatBoost: "+ str(recall_score(y_test, y_pred, average='weighted')))
print("Average F1 of CatBoost: "+ str(f1_score(y_test, y_pred, average='weighted')))
print("F1 of CatBoost for each type of attack: "+ str(f1_score(y_test, y_pred, average=None)))
cb_f1=f1_score(y_test, y_pred, average=None)

# Plot the confusion matrix
cm=confusion_matrix(y_test,y_pred)
f,ax=plt.subplots(figsize=(5,5))
sns.heatmap(cm,annot=True,linewidth=0.5,linecolor="red",fmt=".0f",ax=ax)
plt.xlabel("y_pred")
plt.ylabel("y_true")
plt.show()
print(cm)

In [ ]:
print(cm)

In [ ]:
# Leading model list for each class
model=[]
for i in range(len(lg_f1)):
    if max(lg_f1[i],xg_f1[i],cb_f1[i]) == lg_f1[i]:
        model.append(lg)
    elif max(lg_f1[i],xg_f1[i],cb_f1[i]) == xg_f1[i]:
        model.append(xg)
    else:
        model.append(cb)

In [ ]:
model

In [ ]:
def LCCDE(X_test, y_test, m1, m2, m3):
    i = 0
    t = []
    m = []
    yt = []
    yp = []
    l = []
    pred_l = []
    pro_l = []

    # For each class (normal or a type of attack), find the leader model
    for xi, yi in stream.iter_pandas(X_test, y_test):

        xi2=np.array(list(xi.values()))
        y_pred1 = m1.predict(xi2.reshape(1, -1))      # model 1 (LightGBM) makes a prediction on text sample xi
        y_pred1 = int(y_pred1[0])
        y_pred2 = m2.predict(xi2.reshape(1, -1))      # model 2 (XGBoost) makes a prediction on text sample xi
        y_pred2 = int(y_pred2[0])
        y_pred3 = m3.predict(xi2.reshape(1, -1))      # model 3 (Catboost) makes a prediction on text sample xi
        y_pred3 = int(y_pred3[0])

        p1 = m1.predict_proba(xi2.reshape(1, -1))     # The prediction probability (confidence) list of model 1 
        p2 = m2.predict_proba(xi2.reshape(1, -1))     # The prediction probability (confidence) list of model 2  
        p3 = m3.predict_proba(xi2.reshape(1, -1))     # The prediction probability (confidence) list of model 3  

        # Find the highest prediction probability among all classes for each ML model
        y_pred_p1 = np.max(p1)
        y_pred_p2 = np.max(p2)
        y_pred_p3 = np.max(p3)

        if y_pred1 == y_pred2 == y_pred3: # If the predicted classes of all the three models are the same
            y_pred = y_pred1 # Use this predicted class as the final predicted class

        elif y_pred1 != y_pred2 != y_pred3: # If the predicted classes of all the three models are different
            # For each prediction model, check if the predicted class’s original ML model is the same as its leader model
            if model[y_pred1]==m1: # If they are the same and the leading model is model 1 (LightGBM)
                l.append(m1)
                pred_l.append(y_pred1) # Save the predicted class
                pro_l.append(y_pred_p1) # Save the confidence

            if model[y_pred2]==m2: # If they are the same and the leading model is model 2 (XGBoost)
                l.append(m2)
                pred_l.append(y_pred2)
                pro_l.append(y_pred_p2)

            if model[y_pred3]==m3: # If they are the same and the leading model is model 3 (CatBoost)
                l.append(m3)
                pred_l.append(y_pred3)
                pro_l.append(y_pred_p3)

            if len(l)==0: # Avoid empty probability list
                pro_l=[y_pred_p1,y_pred_p2,y_pred_p3]

            elif len(l)==1: # If only one pair of the original model and the leader model for each predicted class is the same
                y_pred=pred_l[0] # Use the predicted class of the leader model as the final prediction class

            else: # If no pair or multiple pairs of the original prediction model and the leader model for each predicted class are the same
                max_p = max(pro_l) # Find the highest confidence
                
                # Use the predicted class with the highest confidence as the final prediction class
                if max_p == y_pred_p1:
                    y_pred = y_pred1
                elif max_p == y_pred_p2:
                    y_pred = y_pred2
                else:
                    y_pred = y_pred3  
        
        else: # If two predicted classes are the same and the other one is different
            n = mode([y_pred1,y_pred2,y_pred3]) # Find the predicted class with the majority vote
            y_pred = model[n].predict(xi2.reshape(1, -1)) # Use the predicted class of the leader model as the final prediction class
            y_pred = int(y_pred[0]) 

        yt.append(yi)
        yp.append(y_pred) # Save the predicted classes for all tested samples
    return yt, yp

In [ ]:
%%time
# Implementing LCCDE
yt, yp = LCCDE(X_test, y_test, m1 = lg, m2 = xg, m3 = cb)

In [ ]:
# The performance of the proposed lCCDE model
print("Accuracy of LCCDE: "+ str(accuracy_score(yt, yp)))
print("Precision of LCCDE: "+ str(precision_score(yt, yp, average='weighted')))
print("Recall of LCCDE: "+ str(recall_score(yt, yp, average='weighted')))
print("Average F1 of LCCDE: "+ str(f1_score(yt, yp, average='weighted')))
print("F1 of LCCDE for each type of attack: "+ str(f1_score(yt, yp, average=None)))

In [ ]:
# Comparison: The F1-scores for each base model
print("F1 of LightGBM for each type of attack: "+ str(lg_f1))
print("F1 of XGBoost for each type of attack: "+ str(xg_f1))
print("F1 of CatBoost for each type of attack: "+ str(cb_f1))

In [ ]:
df5=X_test
df5["Label"]=yt
df5.head(400)

In [ ]:

df5["predict"]=yp
df5.head()

In [ ]:
df5.to_csv("D:/intrusion output.csv")